In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from utils import *
from preprocess import *
from augmentation import *

## READ CSV

In [2]:
mass_df = pd.read_csv('mass_df.csv')
mass_df

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,img_id,cropped_image_file_path,ROI_mask_file_path,img_path,cropped_path,roi_path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.342386...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.296736...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.296736...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.359308...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.227955...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.227955...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.891800...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.429120...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.429120...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.295360...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.115134...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.115134...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.410524...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.128573...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.128573...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,P_01825,2,RIGHT,MLO,1,mass,LOBULATED,MICROLOBULATED,3,BENIGN_WITHOUT_CALLBACK,3,Mass-Test_P_01825_RIGHT_MLO,Mass-Test_P_01825_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Mass-Test_P_01825_RIGHT_MLO_1/1.3.6.1.4.1.9590...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.404577...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.408076...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.408076...
1692,P_01833,2,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,5,MALIGNANT,5,Mass-Test_P_01833_RIGHT_MLO,Mass-Test_P_01833_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Mass-Test_P_01833_RIGHT_MLO_1/1.3.6.1.4.1.9590...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.355464...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.183250...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.183250...
1693,P_01865,2,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED,4,MALIGNANT,2,Mass-Test_P_01865_LEFT_MLO,Mass-Test_P_01865_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_01865_LEFT_MLO_1/1.3.6.1.4.1.9590....,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.413066...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.358302...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.358302...
1694,P_01912,3,RIGHT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,4,Mass-Test_P_01912_RIGHT_CC,Mass-Test_P_01912_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_01912_RIGHT_CC_1/1.3.6.1.4.1.9590....,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.395806...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.299479...,../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.299479...


In [3]:
mass_df['new_id'] = mass_df['img_id'].str.cat(mass_df['abnormality id'].astype(str), sep='_')
mass_df['new_id']

0         Mass-Training_P_00001_LEFT_CC_1
1        Mass-Training_P_00001_LEFT_MLO_1
2         Mass-Training_P_00004_LEFT_CC_1
3        Mass-Training_P_00004_LEFT_MLO_1
4       Mass-Training_P_00004_RIGHT_MLO_1
                      ...                
1691        Mass-Test_P_01825_RIGHT_MLO_1
1692        Mass-Test_P_01833_RIGHT_MLO_1
1693         Mass-Test_P_01865_LEFT_MLO_1
1694         Mass-Test_P_01912_RIGHT_CC_1
1695        Mass-Test_P_01912_RIGHT_MLO_1
Name: new_id, Length: 1696, dtype: object

### CHECK ID DUPLICATE

In [4]:
duplicates = mass_df['new_id'].duplicated()

if duplicates.any():
    print("There are duplicate values in the 'new_id' column.")
    # Print the rows with duplicate values
    print("Rows with duplicate 'new_id' values:")
    mass_df[duplicates]
else:
    print("There are no duplicate values in the 'new_id' column.")


There are no duplicate values in the 'new_id' column.


## TRAIN TEST SPLIT

In [5]:
from sklearn.model_selection import train_test_split
mass_df.loc[mass_df["pathology"] == "BENIGN_WITHOUT_CALLBACK", "pathology"] = "BENIGN"
train_df, tmp_df = train_test_split(mass_df,train_size= 0.8, stratify=mass_df['pathology'], shuffle=True,random_state=42)
val_df, test_df = train_test_split(tmp_df,train_size= 0.5, stratify=tmp_df['pathology'], shuffle=True,random_state=42)

In [6]:
data_dir = "E:/CBIS-DDSM/Breast_cancer_CBIS-DDSM/data"

train_df.to_csv(f'{data_dir}/ori_train_df.csv', index=False)
val_df.to_csv(f'{data_dir}/ori_val_df.csv', index=False)
test_df.to_csv(f'{data_dir}/ori_test_df.csv', index=False)

## CREATE DATA FOLDERS

In [7]:
aug_dir = f'{data_dir}/aug/'
preprocess_dir = f'{data_dir}/preprocess/'
os.makedirs(aug_dir, exist_ok=True)
os.makedirs(preprocess_dir, exist_ok=True)

## PREPROCESSING

In [8]:
preprocess_folder = "E:/CBIS-DDSM/Breast_cancer_CBIS-DDSM/data/preprocess"
preprocess_train_folder = f"{preprocess_folder}/train/"
preprocess_val_folder = f"{preprocess_folder}/val/"
preprocess_test_folder = f"{preprocess_folder}/test/"

In [9]:
unqualified_img = []

for idx, row in train_df.iterrows():
    check = preprocessing(row, save_dir=preprocess_train_folder)
    if check != None:
        unqualified_img.append(check)

for idx, row in val_df.iterrows():
    check = preprocessing(row, save_dir=preprocess_val_folder)
    if check != None:
        unqualified_img.append(check)

for idx, row in test_df.iterrows():
    check = preprocessing(row, save_dir=preprocess_test_folder)
    if check != None:
        unqualified_img.append(check)

In [10]:
unqualified_img

['../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.425805369410287321022604020872718369142/1-203.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.395876256210876745215504037474275317920/1-298.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.110137861311656868218615358930246510477/1-136.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.345891015013816713918233478951770382675/1-140.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.389840538411334686726458566800452300719/1-164.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.135126056112780974433437774491962300014/1-037.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.423418338111627617941958352860375625032/1-093.jpg',
 '../Kaggle/jpeg/1.3.6.1.4.1.9590.100.1.2.278751293312112492613108122923242306167/1-229.jpg']

## AUGMENTATION

In [11]:
aug_folder = "E:/CBIS-DDSM/Breast_cancer_CBIS-DDSM/data/aug"
aug_train_folder = f"{aug_folder}/train/"
aug_val_folder = f"{aug_folder}/val/"
aug_test_folder = f"{aug_folder}/test/"

apply_augmentations_to_folder(preprocess_train_folder, aug_train_folder)

## CREATE NEW DF FOR NEW IMAGES

In [12]:
preprocess_train_df = create_df_from_subfolder(preprocess_train_folder)
preprocess_val_df = create_df_from_subfolder(preprocess_val_folder)
preprocess_test_df = create_df_from_subfolder(preprocess_test_folder)

aug_train_df = create_df_from_subfolder(aug_train_folder)
# aug_val_df = create_df_from_subfolder(aug_val_folder)
# aug_test_df = create_df_from_subfolder(aug_test_folder)

final_train_df = pd.concat([preprocess_train_df, aug_train_df]).reset_index(drop=True)
# final_val_df = pd.concat([preprocess_val_df, aug_val_df]).reset_index(drop=True)
# final_test_df = pd.concat([preprocess_test_df, aug_test_df]).reset_index(drop=True)

In [13]:
data_dir = "E:/CBIS-DDSM/Breast_cancer_CBIS-DDSM/data"
final_train_df.to_csv(f'{data_dir}/train_df.csv', index=False)
preprocess_val_df.to_csv(f'{data_dir}/val_df.csv', index=False)
preprocess_test_df.to_csv(f'{data_dir}/test_df.csv', index=False)